### useOptimistic() hook

Provides optimistic state that can update to the user immediately, and will be autmatically reverted if there's a problem with the update to the server.

In [ ]:
import ReactDOM from "react-dom/client"
import { useState, useActionState, useOptimistic } from "react"
import { updateNameInDB } from "./api"

function App() {
  const [state, actionFunction, isPending] = useActionState(
    updateName,
    {
      error: null,
      name: JSON.parse(localStorage.getItem("name")) || "Anonymous user"
    }
  )
  
  const [optimisticName, setOptimisticName] = useOptimistic(state.name) ## utilize hook

  async function updateName(prevState, formData) {
    setOptimisticName(formData.get("name")) ## set optimistic name based on data input
    
    ## we assume that our input is correct and we display new name immediately, so 
    ## we can dispaly the name immediately, and in the background update of database will occur,
    ## so we don't have to wait it.
    try {
      const newName = await updateNameInDB(formData.get("name"))
      return { name: newName, error: null }
    } catch (error) {
      return { error, name: prevState.name }
    }
  }

  return (
    <>
      <p className="username">
        Current user: <span>{optimisticName}</span> ## display optimistic Name
      </p>

      <form action={actionFunction}>
        <input
          type="text"
          name="name"
          required
        />
        <button type="submit">Update</button>
        {!isPending && state.error && <p className="error">{state.error.message}</p>}
      </form>
    </>
  )
}

ReactDOM.createRoot(document.getElementById("root")).render(<App />)